In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, lower, regexp_replace, split, to_date

spark= SparkSession.builder.config('spark.ui.port','4040').getOrCreate()

In [2]:
fos = spark.read.csv("file:///N/project/mag/mag-2021-01-05/advanced/FieldsOfStudy.txt",\
                         header = False,\
                         sep=r'\t')
#fos.printSchema()

In [3]:
def clean_text(c):
  c = regexp_replace(c, '"' , '')
  c = regexp_replace(c, '\\\\' , '')
  #c = regexp_replace(c, "(https?\://)\S+", "")
  #c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  #c = split(c, "\\s+") tokenization...
  return c

In [5]:
fos = fos.select(
                 fos._c0.cast("string").alias("fieldOfStudyId"),
                 fos._c1.cast("string").alias("rank"),
                 fos._c2.cast("string").alias("normalizedName"),   
                 clean_text(col("_c3")).alias("displayName"),
                 fos._c4.cast("string").alias("mainType"),
                 fos._c5.cast("int").alias("level"),
                 fos._c6.cast("int").alias("paperCount"),
                 fos._c7.cast("int").alias("paperFamilyCount"),
                 fos._c8.cast("int").alias("citationCount"),
                 to_date(fos._c9, "yyyy-MM-dd").alias("createdDate")
)
fos.show(5)

+--------------+-----+--------------------+--------------------+--------+-----+----------+----------------+-------------+-----------+
|fieldOfStudyId| rank|      normalizedName|         displayName|mainType|level|paperCount|paperFamilyCount|citationCount|createdDate|
+--------------+-----+--------------------+--------------------+--------+-----+----------+----------------+-------------+-----------+
|        417682|16758|           night air|           Night air|    null|    2|        99|              99|          576| 2016-06-24|
|       1443462|15557|         immobiliser|         Immobiliser|    null|    2|       410|             403|         1178| 2016-06-24|
|       1576492|13013|       matrix pencil|       Matrix pencil|    null|    3|      1820|            1777|        26908| 2016-06-24|
|       2657588|13530|combinatorial top...|Combinatorial top...|    null|    3|       571|             541|        20917| 2016-06-24|
|       3079626| 8569|quantum electrody...|Quantum electrody..

In [6]:
%%time

fos.coalesce(64).write.option("header","true") \
                               .option("sep","\t") \
                               .option("quoteAll", False) \
                               .option("emptyValue", None) \
                               .option("nullValue", None)\
                               .mode("overwrite") \
                               .csv('/N/project/mag/mag_jg_2021_update/nodes/fos')               

CPU times: user 6.06 ms, sys: 3.12 ms, total: 9.18 ms
Wall time: 5.89 s
